# Graph Neural Networks with Pytorch

Original Paper: https://arxiv.org/abs/1706.02216    
Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/reddit.py

In [1]:
!pip install pytorch torchvision torchaudio
!pip install torch_scatter
!pip install torch_sparse

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached torchvision-0.14.1-cp310-cp310-win_amd64.whl (1.1 MB)
  Using cached torchaudio-0.13.1-cp310-cp310-win_amd64.whl (2.0 MB)
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch: started
  Running setup.py install for pytorch: finished with status 'error'

[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\wikid\AppData\Local\Temp\pip-install-zgftp7g3\pytorch_26602f29ae6b47ecab2b1c314f9b6d96\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  error: subprocess-exited-with-error
  
  × Running setup.py install for pytorch did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in 


[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import sys

import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.datasets import Reddit
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader


sys.path.append('./')
from utils import *
from model import *
logger = make_logger(name='graphsage_logger')

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = os.path.join(os.getcwd(), '..', 'data', 'Reddit')
dataset = Reddit(path)
data = dataset[0]

In [4]:
train_loader_size = [25, 10]
subgraph_loader_size = [-1]
num_worker = 8
epoch_num = 6
batch = 1024

In [5]:
logger.info(f"Node Feature Matrix Info: # Nodes: {data.x.shape[0]}, # Node Features: {data.x.shape[1]}")

logger.info(f"Edge Index Shape: {data.edge_index.shape}")
logger.info(f"Edge Weight: {data.edge_attr}")

print(data.train_mask.sum().item())

2023-03-09 20:06:06,185 - graphsage_logger - Node Feature Matrix Info: # Nodes: 232965, # Node Features: 602
2023-03-09 20:06:06,187 - graphsage_logger - Edge Index Shape: torch.Size([2, 114615892])
2023-03-09 20:06:06,188 - graphsage_logger - Edge Weight: None


153431


In [6]:
train_loader = NeighborSampler(
    data.edge_index, node_idx=data.train_mask,
    sizes=train_loader_size, batch_size=batch, shuffle=True, num_workers=num_worker)

subgraph_loader = NeighborSampler(
    data.edge_index, node_idx=None,
    sizes=subgraph_loader_size, batch_size=batch, shuffle=False, num_workers=num_worker)

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.NeighborSampler' is deprecated, use 'loader.NeighborSampler' instead
  warnings.warn(out)


In [7]:
batch_size, n_id, adjs = next(iter(train_loader))

logger.info(f"Current Batch Size: {batch_size}")
logger.info(f"현재 Subgraph에서 사용된 모든 node id의 개수: {n_id.shape[0]}")
logger.info(f"Layer의 수: {len(adjs)}")

A = adjs[1].size[0] - batch_size
B = adjs[0].size[0] - A - batch_size

logger.info(f"진행 방향: {B}개의 2-hop neighbors ->"
            f"{A}개의 1-hop neighbors -> {batch_size}개의 Head Nodes")

2023-03-09 20:07:48,346 - graphsage_logger - Current Batch Size: 1024
2023-03-09 20:07:48,349 - graphsage_logger - 현재 Subgraph에서 사용된 모든 node id의 개수: 107146
2023-03-09 20:07:48,350 - graphsage_logger - Layer의 수: 2
2023-03-09 20:07:48,351 - graphsage_logger - 진행 방향: 85205개의 2-hop neighbors ->20917개의 1-hop neighbors -> 1024개의 Head Nodes


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(dataset.num_features, 256, dataset.num_classes)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)

x = data.x.to(device)
y = data.y.squeeze().to(device)

In [9]:
def train(epoch):
    model.train()

    pbar = tqdm(total=int(data.train_mask.sum()))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = 0
    for batch_size, n_id, adjs in train_loader:
        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out = model(x[n_id], adjs)
        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        pbar.update(batch_size)

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / int(data.train_mask.sum())
    return loss, approx_acc

@torch.no_grad()
def test():
    model.eval()
    out = model.inference(x, subgraph_loader, device)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    results = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
    return results

In [10]:
# for epoch in range(1, epoch_num):
#     loss, acc = train(epoch)
#     print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
#     scheduler.step()
#     train_acc, val_acc, test_acc = test()
#     print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

In [11]:
best_val_acc = 0
best_model = None
best_epoch = 0
patience = 2
counter = 0

for epoch in range(1, epoch_num+1):
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
    scheduler.step()
    
    train_acc, val_acc, test_acc = test()
    print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    # Early Stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch
        best_model = model.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping: validation accuracy did not improve for {patience} epochs.')
            break

print(f'Best validation accuracy of {best_val_acc:.4f} was achieved at epoch {best_epoch}.')
model.load_state_dict(best_model)
test_acc = test()
print(f'Test accuracy of {test_acc[0]:.4f} was achieved at epoch {best_epoch}.')

Epoch 01: 100%|██████████| 153431/153431 [03:11<00:00, 801.66it/s] 


Epoch 01, Loss: 0.5285, Approx. Train: 0.8914


Evaluating: 100%|██████████| 465930/465930 [07:24<00:00, 1048.92it/s]


Train: 0.9518, Val: 0.9496, Test: 0.9501


Epoch 02: 100%|██████████| 153431/153431 [03:38<00:00, 701.15it/s]


...

Epoch 06, Loss: 0.1919, Approx. Train: 0.9542


Evaluating: 100%|██████████| 465930/465930 [06:36<00:00, 1175.32it/s]


Train: 0.9716, Val: 0.9624, Test: 0.9617
Best validation accuracy of 0.9624 was achieved at epoch 5.


Evaluating: 100%|██████████| 465930/465930 [06:35<00:00, 1179.09it/s]

Test accuracy of 0.9716 was achieved at epoch 5.


In [18]:
torch.save(best_model, 'best_model.pth')